## Nova Canvas Fine-Tuning for Character Consistency

This notebook demonstrates how to fine-tune Amazon Nova Canvas to create character-consistent storyboards using images from the animated short film \"Picchu\".

## Introduction

In this notebook, we'll walk through the process of fine-tuning Amazon Nova Canvas to maintain visual consistency for specific characters (Mayu and her mom) across multiple generated images. This approach allows for more precise control over character appearance than prompt engineering alone.

### Prerequisites
- AWS account with access to Amazon Bedrock
- Appropriate IAM permissions for Bedrock, S3, and related services
- This notebook must be run in the `us-east-1` AWS region

<div class="alert alert-info">
<b>INFO:</b> At the time of development, Amazon Nova Canvas model is only available in `us-east-1`
</div>

### What You'll Learn
- How to prepare training data from existing character images
- How to configure and run a fine-tuning job on Amazon Nova Canvas
- How to generate character-consistent storyboard frames with your fine-tuned model

You can watch the original \"Picchu\" animated short film here: [Picchu on YouTube](https://www.youtube.com/watch?v=XfyJbkRV_Eo)

### Setup

First, let's install the required dependencies for this notebook. These packages will help us with image processing, AWS interactions, and visualization.
 

In [ ]:
!pip install -r requirements.txt

### Initialize

Now we'll set up our AWS environment by initializing the necessary clients and defining key variables. This includes:
- Setting up boto3 clients for Bedrock, S3, IAM, and STS
- Defining our S3 bucket and prefix for storing training data
- Setting the base model ID for fine-tuning
- Creating a directory for our training images

In [ ]:
import boto3
import sagemaker
from sagemaker.utils import name_from_base
import time
import json
from image_processing import process_folders, upload_to_s3
import os
import shutil

region = "us-east-1"
boto_session = boto3.Session(region_name=region)
sess = sagemaker.Session(boto_session=boto_session)

if sess.default_bucket(): 
    bucket = sess.default_bucket()
else:
    bucket = "XXXXXXXXXXXXX" # Use your own

prefix = "picchu-canvas/images"


# Initialize Boto3 Clients
bedrock = boto_session.client('bedrock')
bedrock_runtime = boto_session.client('bedrock-runtime')
s3 = boto_session.client('s3')
iam_client = boto_session.client('iam')
sts_client = boto_session.client('sts')

# Account and region info
account_id = sts_client.get_caller_identity()["Account"]

# Base model id for fine-tuning
model_id = 'amazon.nova-canvas-v1:0'

image_dir = "picchu_images"

### Prepare Local Directory

We'll create a clean directory to store our training images. If the directory already exists, we'll remove it and create a fresh one to avoid any conflicts with previous files.

In [ ]:
def create_or_replace_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # Remove the existing folder and its contents
    os.makedirs(folder_path)         # Create a new, empty folder

create_or_replace_folder(image_dir)

## Download images

In this step, we'll download a pre-prepared set of images from the \"Picchu\" animated short film. These images will serve as our training data for fine-tuning the Nova Canvas model to consistently generate the main character Mayu and her mom.
  

In [ ]:
!wget --no-check-certificate https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/3c3519c9-93dc-404d-87f7-4d9bde05f265/picchu_images.zip

!unzip picchu_images.zip

### Prepare the images for fine tuning

Now we'll process the downloaded images and prepare them for fine-tuning. This function will:
1. Upload the images to our S3 bucket
2. Generate a manifest file that pairs each image with a descriptive caption
3. The manifest file follows the required format for Amazon Nova Canvas fine-tuning

For more information on manifest file requirements, see the [Amazon Bedrock documentation on fine-tuning](https://docs.aws.amazon.com/bedrock/latest/userguide/custom-models.html).
   

### Process the downloaded images

Now we'll process the images from the downloaded zip file. The `process_folders` function will:
1. Extract images from the specified directories
2. Generate appropriate captions for each image
3. Upload the images to S3 and prepare the data structure for our manifest file

This step is crucial for preparing our training data in the format required by Amazon Nova Canvas.

In [ ]:
updated_data = process_folders([image_dir], bucket, prefix)

In [ ]:
output_file = f'{prefix.split('-')[0]}_manifest.jsonl'
with open(output_file, 'w') as f:
    for item in updated_data:
        item_filtered = {d:item[d] for d in item if d != 'id'}
        f.write(json.dumps(item_filtered) + '\n')
print(f"{output_file} processed completed!")

### Preview the manifest file

Let's examine the first few entries of our manifest file to understand its structure. Each line contains a JSON object with:
- `image-ref`: The S3 path to the image
- `caption`: A detailed description of the image that helps the model learn the character's appearance and style

In [ ]:
!head -n 5 {output_file}

### Upload manifest to S3

Now we'll upload our manifest file to S3. This file will be used by the fine-tuning job to locate and process our training images along with their captions.
   

In [ ]:
training_path = upload_to_s3(output_file, bucket, prefix.replace("images", "manifests"))
training_path

## Train Custom Model Using Bedrock

Now we'll begin the process of fine-tuning the Amazon Nova Canvas model using our prepared training data. This involves several steps:
1. Creating the necessary IAM roles and policies
2. Configuring the fine-tuning job parameters
3. Submitting the job to Amazon Bedrock
4. Monitoring the job progress

### Fine tune job preparation - Creating role and policies requirements

We will now prepare the necessary IAM role for the fine-tune job. This includes creating the policies required to run customization jobs with Amazon Bedrock.

### Create Trust relationship
This JSON object defines the trust relationship that allows the Bedrock service to assume a role that will give it the ability to interact with other required AWS services. The conditions set restrict the assumption of the role to a specific account ID and a specific component of the Bedrock service (model_customization_jobs).
  

In [ ]:
# This JSON object defines the trust relationship that allows the bedrock service to assume a role that will give it the ability to talk to other required AWS services. The conditions set restrict the assumption of the role to a specfic account ID and a specific component of the bedrock service (model_customization_jobs)
ROLE_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Principal": {{
                "Service": "bedrock.amazonaws.com"
            }},
            "Action": "sts:AssumeRole",
            "Condition": {{
                "StringEquals": {{
                    "aws:SourceAccount": "{account_id}"
                }},
                "ArnEquals": {{
                    "aws:SourceArn": "arn:aws:bedrock:{region}:{account_id}:model-customization-job/*"
                }}
            }}
        }}
    ]
}}
"""

### Create S3 access policy

This JSON object defines the permissions of the role we want bedrock to assume to allow access to the S3 bucket that we created that will hold our fine-tuning datasets and allow certain bucket and object manipulations.

In [ ]:
ACCESS_POLICY_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:DeleteObject",
                "s3:PutObject",
                "s3:GetObject",
                "s3:GetBucketAcl",
                "s3:GetBucketNotification",
                "s3:ListBucket",
                "s3:PutBucketNotification"
            ],
            "Resource": [
                "arn:aws:s3:::{bucket}",
                "arn:aws:s3:::{bucket}/*"
            ]
        }}
    ]
}}"""

### Create IAM role and attach policies

Let's now create the IAM role with the created trust policy and attach the s3 policy to it

In [ ]:
role_name = name_from_base(f"FineTuning-{prefix.split('/')[-1]}")
s3_bedrock_ft_access_policy = f"{role_name}-policy"

In [ ]:
response = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=ROLE_DOC,
    Description="Role for Bedrock to access S3 for finetuning",
)

In [ ]:
role_arn = response["Role"]["Arn"]
response = iam_client.create_policy(
    PolicyName=s3_bedrock_ft_access_policy,
    PolicyDocument=ACCESS_POLICY_DOC,
)
policy_arn = response["Policy"]["Arn"]
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn,
)
time.sleep(30)

### Create a Customization Job

Now that we have all the requirements in place, let's create the fine-tuning job with the Titan Image Generator model.

To do so, we need to set the model **hyperparameters** for `stepCount`, `batchSize` and `learningRate` and provide the path to your training data

In [ ]:
# Set parameters
model_name =  f"{prefix.split('/')[0]}-v0"
roleArn = role_arn
jobName = name_from_base(model_name)
customModelName = jobName
hyperParameters = {
        "stepCount": "14000",
        "batchSize": "64",
        "learningRate": "0.000001",
    }
trainingDataConfig = {"s3Uri": training_path}
outputDataConfig = {"s3Uri": f"s3://{bucket}/{prefix}"}

# Create job
response_ft = bedrock.create_model_customization_job(
    jobName=jobName,
    customModelName=customModelName,
    roleArn=roleArn,
    baseModelIdentifier=model_id,
    hyperParameters=hyperParameters,
    trainingDataConfig=trainingDataConfig,
    outputDataConfig=outputDataConfig
)

jobArn = response_ft.get('jobArn')
print(jobArn)

### Waiting until customization job is completed
Once the customization job is finished, you can check your existing custom model(s) and retrieve the modelArn of your fine-tuned model.

<div class=\"alert alert-block alert-warning\">
    <b>Warning:</b> The model customization job can take hours to run. With 12000 steps, 0.000001 learning rate, 64 of batch size and x images, it takes around 4 hours to complete
</div>

### Set Job Name

The code below allows you to check the status of an existing job.

<div class="alert alert-warning">
<b>WARNING:</b> This job can take up to 12 hours to complete. Please be patient, you can always comeback and continue the subsequent steps.
</div>

In [ ]:
# check model customization status
status = bedrock.list_model_customization_jobs(
    nameContains=jobName
)["modelCustomizationJobSummaries"][0]["status"]

print(status)

Once Complete, get the new `customModelARN` using the code below, or you can manually set the parameter by copying the `customModelARN` from the console. 

In [ ]:
custom_model_arn = bedrock.list_model_customization_jobs(
    nameContains=jobName
)["modelCustomizationJobSummaries"][0]["customModelArn"]

In [ ]:
#custom_model_arn = ""

## Host Fine-tuned Model With Provisioned Throughput (PT)

You will need to create provisioned throughput to be able to evaluate the model performance. You can do so through the console or use the following api call.

<div class="alert alert-warning">
<b>WARNING:</b> Creating provisioned throughput will take 30mins to complete.
</div>

In [ ]:
custom_model_name = name_from_base(model_name)

# Create the provision throughput job and retrieve the provisioned model id
provisioned_model_id = bedrock.create_provisioned_model_throughput(
    modelUnits=1,
    # create a name for your provisioned throughput model
    provisionedModelName=custom_model_name, 
    modelId=custom_model_arn
)['provisionedModelArn']

In [ ]:
%%time
# check provisioned throughput job status
import time
status_provisioning = bedrock.get_provisioned_model_throughput(provisionedModelId = provisioned_model_id)['status'] 
while status_provisioning == 'Creating':
    time.sleep(60)
    status_provisioning = bedrock.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id)['status']
    print(status_provisioning)

### Set Provisioned Model ID

Similar to the job name, if you're returning to this notebook after creating a provisioned throughput, you can set the provisioned model ID manually here. This allows you to use an existing provisioned model without waiting for a new one to be created.

In [ ]:
# provisioned_model_id = "<ENTER-YOUR-PT-ARN>"

## Test Fine-tuned Model
We will now run some model experiments using the bedrock-runtime client with the invoke_model function to invoke both fine-tuned and pre-trained models.

To invoke the provisioned custom model, notice you will need to run the previous step (create provisioned throughput) before proceeding.

In [ ]:
import json
import io
from PIL import Image
import base64

def decode_base64_image(img_b64):
    return Image.open(io.BytesIO(base64.b64decode(img_b64)))
    
def generate_image(prompt,
                   negative_prompt="text, ugly, blurry, distorted, low quality, pixelated, watermark, text, deformed", 
                   num_of_images=3,
                   seed=1):
    """
    Generate an image using Amazon Nova Canvas.
    """

    image_gen_config = {
            "numberOfImages": num_of_images,
            "quality": "premium",
            "width": 1024,  # Maximum resolution 2048 x 2048
            "height": 1024,  # 1:1 ratio
            "cfgScale": 8.0,
            "seed": seed,
        }

    # Prepare the request body
    request_body = {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,
            "negativeText": negative_prompt,  # List things to avoid
        },
        "imageGenerationConfig": image_gen_config
    } 

    response = bedrock_runtime.invoke_model(
        modelId=provisioned_model_id,
        body=json.dumps(request_body)
    )

    # Parse the response
    response_body = json.loads(response['body'].read())

    if "images" in response_body:
        # Extract the image
        return [decode_base64_image(img) for img in response_body['images']]
    else:
        print(response_body)
        return

In [ ]:
prompt = "Mayu smiling with a mistic forest in the background"
# prompt = "Close-up of Mayu gestures towards a cute, fluffy white baby llama with big curious eyes."
# prompt = "Mayu is sitting on the ground with fence of rope around her. She has a confused expression. A fluffy white baby llama prances near her. The background shows hills and sky."
# prompt = "Mayu stands in front of a wooden fence. A cute fluffy white baby llama stands triumphantly in the foreground. Small bags are scattered on the ground near the llama. A comical scene in the Peruvian Andes."
# prompt = "Mayu playing a flute. Beside her, a fluffy white baby llama with its big curious eyes focused on Mayu. The background showcases Andean mountains."
# prompt = "An Andean village scene with traditional buildings. In the center, Mayu stands confidently playing a wooden flute. Next to her, a cute white baby llama dances, carrying a small woven bag in its mouth"
# prompt = "Mayu standing proudly at the entrance of a simple school building. Her face beams with a wide smile, expressing pride and accomplishment."
# prompt = "Mayu face shows a mix of nervousness and determination. Mommy kneels beside her, gently holder her. A landscape is visible in the background."
# prompt = "A steep cliff face with a long wooden ladder extending downwards. Halfway down the ladder is Mayu with a determined expression on her face. Mayu’s small hands grip the sides of the ladder tightly as she carefully places her feet on each rung. The surrounding environment shows a rugged, mountainous landscape."

In [ ]:
import matplotlib.pyplot as plt
import random
import uuid

seed = random.randint(1, 858993459)
print(f"seed: {seed}")

images = generate_image(prompt=prompt, seed=seed)

if images:
    n = len(images)
    cols = min(3, n)  # number of columns
    rows = (n + cols - 1) // cols
    
    plt.figure(figsize=(cols * 5, rows * 5))
    for i, img in enumerate(images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

## Clean Up

<div class="alert alert-danger">
<b>WARNING:</b> After you are done, please remove provision throughput and fine-tuned model to avoid additional charges
</div>

Delete Provisioned Throughput

In [ ]:
response = bedrock.delete_provisioned_model_throughput(
    provisionedModelId=provisioned_model_id
)

Delete Custom Model

In [ ]:
response = bedrock.delete_custom_model(
    modelIdentifier=custom_model_arn
)